In [8]:
token="9xibe_4mK4nZBju5odStmGgzGsTbmW10-CaXpcOvmOdXkLehlQOiDxXibiIOV9fRB7m33pM1_r26G1IJNQPzvg=="

In [28]:
# !pip install influxdb-client

In [41]:
from confluent_kafka import Consumer, KafkaException
import pandas as pd
import json


def msg_dict(data_string):
    # Split the string into key-value pairs
    key_value_pairs = data_string.split(',')

    # Create a dictionary from the key-value pairs
    data_dict = {}
    for pair in key_value_pairs:
        try:
            key, value = pair.split(':')
            data_dict[key] = value
        except:
            key, value = pair.split('e:')
            data_dict[key+'e'] = value

    return data_dict



def kafka_consumer_dataframe():
    consumer_conf = {'bootstrap.servers': 'localhost:9092', 
                     'group.id': 'processed-data-group00',
                     'auto.offset.reset': 'earliest' }
    c = Consumer(consumer_conf)
    c.subscribe(['csv_topic'])
    list_msg=list()
    try:
        while True:
            msg = c.poll(1.0)
            if msg is None:
                continue
            if msg.error():
                if msg.error().code() == KafkaException._PARTITION_EOF:
                    continue
                else:
                    print(msg.error())
                    break

            list_msg.append(msg_dict(msg.value().decode('utf-8')))
            
            if len(list_msg) % 1000 == 0:
                print(f"Rows Consumed: {len(list_msg)}")
    except KeyboardInterrupt:
        pass

    finally:
        c.close()
    print(f"Total:{len(list_msg)}")
    return list_msg


In [42]:
stream=kafka_consumer_dataframe()

Rows Consumed: 1000
Rows Consumed: 2000
Rows Consumed: 3000
Rows Consumed: 4000
Rows Consumed: 5000
Rows Consumed: 6000
Rows Consumed: 7000
Rows Consumed: 8000
Rows Consumed: 9000
Rows Consumed: 10000
Rows Consumed: 11000
Rows Consumed: 12000
Rows Consumed: 13000
Rows Consumed: 14000
Rows Consumed: 15000
Rows Consumed: 16000
Rows Consumed: 17000
Rows Consumed: 18000
Rows Consumed: 19000
Rows Consumed: 20000
Rows Consumed: 21000
Rows Consumed: 22000
Rows Consumed: 23000
Rows Consumed: 24000
Rows Consumed: 25000
Rows Consumed: 26000
Rows Consumed: 27000
Rows Consumed: 28000
Rows Consumed: 29000
Rows Consumed: 30000
Rows Consumed: 31000
Rows Consumed: 32000
Rows Consumed: 33000
Rows Consumed: 34000
Rows Consumed: 35000
Rows Consumed: 36000
Rows Consumed: 37000
Rows Consumed: 38000
Rows Consumed: 39000
Rows Consumed: 40000
Rows Consumed: 41000
Rows Consumed: 42000
Rows Consumed: 43000
Rows Consumed: 44000
Rows Consumed: 45000
Rows Consumed: 46000
Rows Consumed: 47000
Rows Consumed: 48000
R

In [43]:
stream

[{'Datetime': '1/1/2017 0:00',
  'Temperature': '6.559',
  'Humidity': '73.8',
  'WindSpeed': '0.083',
  'GeneralDiffuseFlows': '0.051',
  'DiffuseFlows': '0.119',
  'PowerConsumption_Zone1': '34055.6962',
  'PowerConsumption_Zone2': '16128.87538',
  'PowerConsumption_Zone3': '20240.96386'},
 {'Datetime': '1/1/2017 0:10',
  'Temperature': '6.414',
  'Humidity': '74.5',
  'WindSpeed': '0.083',
  'GeneralDiffuseFlows': '0.07',
  'DiffuseFlows': '0.085',
  'PowerConsumption_Zone1': '29814.68354',
  'PowerConsumption_Zone2': '19375.07599',
  'PowerConsumption_Zone3': '20131.08434'},
 {'Datetime': '1/1/2017 0:20',
  'Temperature': '6.313',
  'Humidity': '74.5',
  'WindSpeed': '0.08',
  'GeneralDiffuseFlows': '0.062',
  'DiffuseFlows': '0.1',
  'PowerConsumption_Zone1': '29128.10127',
  'PowerConsumption_Zone2': '19006.68693',
  'PowerConsumption_Zone3': '19668.43373'},
 {'Datetime': '1/1/2017 0:30',
  'Temperature': '6.121',
  'Humidity': '75',
  'WindSpeed': '0.083',
  'GeneralDiffuseFlows

In [44]:
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

# token = os.environ.get("INFLUXDB_TOKEN")
org = "ELTE"
url = "http://localhost:8086"

write_client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)

In [45]:
stream[18]

{'Datetime': '1/1/2017 3:00',
 'Temperature': '5.169',
 'Humidity': '77.9',
 'WindSpeed': '0.083',
 'GeneralDiffuseFlows': '0.066',
 'DiffuseFlows': '0.108',
 'PowerConsumption_Zone1': '21107.8481',
 'PowerConsumption_Zone2': '13535.56231',
 'PowerConsumption_Zone3': '15140.24096'}

In [97]:
# !pip install tqdm

In [55]:
from tqdm import tqdm

In [50]:
bucket="DREAD"

In [56]:
write_api = write_client.write_api(write_options=SYNCHRONOUS)
   
for data_point in tqdm(stream):
    timestamp = int(time.mktime(time.strptime(data_point['Datetime'], '%m/%d/%Y %H:%M')))
    
    point = (
        Point("consumption")  
        .tag("dataset", "ingestion") 
        .field("Temperature", float(data_point['Temperature']))
        .field("Humidity", float(data_point['Humidity']))
        .field("WindSpeed", float(data_point['WindSpeed']))
        .field("GeneralDiffuseFlows", float(data_point['GeneralDiffuseFlows']))
        .field("DiffuseFlows", float(data_point['DiffuseFlows']))
        .field("PowerConsumption_Zone1", float(data_point['PowerConsumption_Zone1']))
        .field("PowerConsumption_Zone2", float(data_point['PowerConsumption_Zone2']))
        .field("PowerConsumption_Zone3", float(data_point['PowerConsumption_Zone3']))
        .time(timestamp, WritePrecision.S)
    )

    # Write the data point to InfluxDB
    write_api.write(bucket=bucket, org='ELTE', record=point)

    time.sleep(0.05)

  0%|          | 0/52416 [00:00<?, ?it/s]

100%|██████████| 52416/52416 [50:15<00:00, 17.38it/s]


In [82]:
query_api = write_client.query_api()

query = """from(bucket: "DREAD")
 |> range(start: 2016-01-01T00:00:00Z, stop: 2019-01-01T00:00:00Z)
 |> filter(fn: (r) => r._measurement == "consumption")
 """
# query2="""SELECT * FROM "consumption" """
# Run the query
result = query_api.query(org='ELTE', query=query)


In [96]:
for record in result[5].records[:50]:
    print(record)

FluxRecord() table: 5, {'result': '_result', 'table': 5, '_start': datetime.datetime(2016, 1, 1, 0, 0, tzinfo=tzutc()), '_stop': datetime.datetime(2019, 1, 1, 0, 0, tzinfo=tzutc()), '_time': datetime.datetime(2016, 12, 31, 23, 0, tzinfo=tzutc()), '_value': 20240.96386, '_field': 'PowerConsumption_Zone3', '_measurement': 'consumption', 'dataset': 'ingestion'}
FluxRecord() table: 5, {'result': '_result', 'table': 5, '_start': datetime.datetime(2016, 1, 1, 0, 0, tzinfo=tzutc()), '_stop': datetime.datetime(2019, 1, 1, 0, 0, tzinfo=tzutc()), '_time': datetime.datetime(2016, 12, 31, 23, 10, tzinfo=tzutc()), '_value': 20131.08434, '_field': 'PowerConsumption_Zone3', '_measurement': 'consumption', 'dataset': 'ingestion'}
FluxRecord() table: 5, {'result': '_result', 'table': 5, '_start': datetime.datetime(2016, 1, 1, 0, 0, tzinfo=tzutc()), '_stop': datetime.datetime(2019, 1, 1, 0, 0, tzinfo=tzutc()), '_time': datetime.datetime(2016, 12, 31, 23, 20, tzinfo=tzutc()), '_value': 19668.43373, '_fiel